In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [2]:
#loading the model
model = load_model("simple_rnn_colab.h5")
#weights of models
#model.get_weights()

In [3]:
max_features = 10000 # Only the top 10,000 words are considered, vocabulary size
max_len = 500 # Maximum review length

# Load word index mapping from IMDB dataset
word_index = imdb.get_word_index()

# Reverse the word index to map integers back to words
reverse_word_index = { value:key for key,value in word_index.items()}

#function to decode the review
def decoded_review(encoded_review):
    return " ".join([reverse_word_index.get(i-3, "?") for i in encoded_review])


#function to preprocess the user input into a padded sequence suitable for model prediction
def preprocessed_text(text, max_features=10000, max_len=500):
    words = text.lower().split()
    
    # Encode each word using the IMDB word index
    encoded_review = [word_index.get(word,2)+3 for word in words] # 2 is for unknown words

    # Pad the sequence to a fixed length
    padded_review = pad_sequences([encoded_review], maxlen=max_len)
    
    # Clip to ensure no index exceeds the embedding input range
    padded_review = padded_review.clip(0, max_features - 1)

    return padded_review

In [4]:
#prediction function
def predict_sentiment(review):
    preprocessed_review = preprocessed_text(review)
    
    prediction = model.predict(preprocessed_review)

    sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"

    return sentiment, prediction[0][0]

In [5]:
#user input and prediction

example = "the movie was awesom and the acting was great and plot was thrilling even acting was good, connected to the audiance"
sentiment, score = predict_sentiment(example)

print(f"The movie review is: ' {example}', \n The sentiment is: {sentiment} \n prediction score: {score}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
The movie review is: ' the movie was awesom and the acting was great and plot was thrilling even acting was good, connected to the audiance', 
 The sentiment is: Positive 
 prediction score: 0.89035964012146
